In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from functools import reduce
import matplotlib as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#import csv of oscar winners
cast_awards_2017=pd.read_csv("../DataSources/Oscars2017.csv")

cast_awards_2017

,Title,Best Director,Best Actor,Best Actress,Best Supporting Actor,Best Supporting Actress
0,The Shape of Water,1,0,0,0,0
1,Darkest Hour,0,1,0,0,0
2,Dunkirk,0,0,0,0,0
3,Phantom Thread,0,0,0,0,0
4,The Post,0,0,0,0,0
5,Call Me by Your Name,0,0,0,0,0
6,Lady Bird,0,0,0,0,0
7,"Three Billboards Outside Ebbing, Missouri",0,0,1,1,0
8,Get Out,0,0,0,0,0


In [3]:
#OMBD Calls - done on movies list
theMoviesDF_OMBD = pd.read_csv('../DataSources/theMoviesDF2017_OMBD.csv')
theMoviesDF_OMBD.head()

,Actors,Awards,Box Office,Country,Director,Genre,Rated,Title,Writer(s),Year,imdbRating
0,"Sally Hawkins, Michael Shannon, Richard Jenkin...",Won 4 Oscars. Another 101 wins & 293 nominations.,"$57,393,976",USA,Guillermo del Toro,"Adventure, Drama, Fantasy, Romance, Thriller",R,The Shape of Water,"Guillermo del Toro (screenplay by), Vanessa Ta...",2017,7.4
1,"Gary Oldman, Kristin Scott Thomas, Ben Mendels...",Won 2 Oscars. Another 40 wins & 63 nominations.,"$28,086,332","UK, USA",Joe Wright,"Biography, Drama, History, War",PG-13,Darkest Hour,Anthony McCarten,2017,7.4
2,"Fionn Whitehead, Damien Bonnard, Aneurin Barna...",Won 3 Oscars. Another 51 wins & 197 nominations.,"$188,042,171","UK, Netherlands, France, USA",Christopher Nolan,"Action, Drama, History, Thriller, War",PG-13,Dunkirk,Christopher Nolan,2017,7.9
3,"Vicky Krieps, Daniel Day-Lewis, Lesley Manvill...",Won 1 Oscar. Another 46 wins & 93 nominations.,"$947,190","USA, UK",Paul Thomas Anderson,"Drama, Romance",R,Phantom Thread,Paul Thomas Anderson,2017,7.5
4,"Meryl Streep, Tom Hanks, Sarah Paulson, Bob Od...",Nominated for 2 Oscars. Another 16 wins & 97 n...,"$80,369,969","USA, UK",Steven Spielberg,"Biography, Drama, History, Thriller",PG-13,The Post,"Liz Hannah, Josh Singer",2017,7.2


In [4]:
#TMDB Calls
theMoviesDF_TMDB=pd.read_csv('../DataSources/theMoviesDF2017_TMDB.csv')
theMoviesDF_TMDB.head()

,Title,Budget,Popularity,Release Date,Revenue,Runtime,Vote_average,Vote_count
0,The Shape of Water,19500000,21.814,2017-12-01,194989655,123,7.3,6513
1,Darkest Hour,30000000,14.713,2017-11-22,147568410,125,7.3,2476
2,Dunkirk,100000000,21.118,2017-07-19,525573161,107,7.4,8442
3,Phantom Thread,35000000,13.060,2017-12-25,46261438,131,7.2,1449
4,The Post,50000000,13.776,2017-12-22,174503458,116,7.0,2243


In [5]:
#fix any title discrepancies between OMBD and TMDB dataframes by replacing incorrect OMBD names with TMDB names
#OMBD does not have data on the following movies: Precious: Based on the Novel 'Push' by Sapphire, The Postman (Il Postino), Cries and Whispers, Nashville 
bad_names_OMBD=[]
correct_names_TMDB=[]

for i in theMoviesDF_OMBD["Title"].values:
    if i in theMoviesDF_TMDB["Title"].values:
        pass
    else:
        bad_names_OMBD.append(i)


for i in theMoviesDF_TMDB["Title"].values:
    if i in theMoviesDF_OMBD["Title"].values:
        pass
    else:
        correct_names_TMDB.append(i)

bad_names_OMBD

[]

In [6]:
correct_names_TMDB

[]

In [7]:
theMoviesDF_OMBD.shape

(9, 11)

In [8]:
theMoviesDF_TMDB.shape

(9, 8)

# EXTRACTING INFO FROM API CALLS

In [9]:
#DataFrame by Genre (OMBD)
genreMovies=theMoviesDF_OMBD[['Title', 'Genre']]

for i in range(len(genreMovies['Genre'])):
    x=genreMovies['Genre'][i].split(", ")
    genreMovies['Genre'][i]=x

s=genreMovies.apply(lambda x:pd.Series(x['Genre']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Genre'
BP_genres=genreMovies.drop('Genre', axis=1).join(s)
BP_genres=pd.get_dummies(BP_genres, columns=['Genre'])
BP_genres=BP_genres.groupby('Title').sum()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
#Dataframe by IMBD rating (OMBD)
BP_IMBD=theMoviesDF_OMBD[['Title','imdbRating']]

#Dataframe by Rating (OMBD)
BP_Rating_qual=theMoviesDF_OMBD[['Title','Rated']]
#consolidate certain ratings
BP_Rating_qual['Rated']=BP_Rating_qual['Rated'].replace({'GP':'PG',
                                                        'M/PG': 'PG',
                                                        'TV-MA': 'R',
                                                        'TV-PG': 'PG',
                                                        'PASSED':'APPROVED',
                                                        'NOT RATED': 'UNRATED'})
BP_Rating=pd.get_dummies(BP_Rating_qual, columns=['Rated'])

#Dataframe by Country (OMBD)
BP_Country_qual=theMoviesDF_OMBD[['Title','Country']]
BP_Country=pd.get_dummies(BP_Country_qual, columns=['Country'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [11]:
#Dataframe by Revenue and Budget (TMDB)
BP_Revenue_Budget=theMoviesDF_TMDB[['Title','Revenue', 'Budget']]

#Dataframe by Popularity (TMDB)
BP_Popularity = theMoviesDF_TMDB[['Title','Popularity']]

#Dataframe by Run Time (TMDB)
BP_Runtime = theMoviesDF_TMDB[['Title','Runtime']]

In [12]:
#Dataframe for Release Month (TMDB)
BP_RelMonth_qual = theMoviesDF_TMDB[['Title','Release Date']]
for i in range(len(BP_RelMonth_qual['Release Date'])):
    x=BP_RelMonth_qual['Release Date'][i].split("-")
    BP_RelMonth_qual['Release Date'][i]=x[1]

BP_RelMonth=pd.get_dummies(BP_RelMonth_qual, columns=['Release Date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
#create a spreadsheet of all info (cannot have string as values)
theMoviesDF=pd.merge(cast_awards_2017, BP_IMBD, on= 'Title', how='inner')
dfs=[theMoviesDF, BP_Popularity, BP_Rating, BP_RelMonth, BP_Revenue_Budget, BP_Runtime, BP_genres]
theMoviesDF= reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title'], how='left'), dfs)
theMoviesDF.fillna(0, inplace=True)
theMoviesDF.to_csv('../DataSources/theMovieFeatures2017.csv', index=False)
theMoviesDF

,Title,Best Director,Best Actor,Best Actress,Best Supporting Actor,Best Supporting Actress,imdbRating,Popularity,Rated_PG-13,Rated_R,...,Genre_Comedy,Genre_Crime,Genre_Drama,Genre_Fantasy,Genre_History,Genre_Horror,Genre_Mystery,Genre_Romance,Genre_Thriller,Genre_War
0,The Shape of Water,1,0,0,0,0,7.4,21.814,0,1,...,0,0,1,1,0,0,0,1,1,0
1,Darkest Hour,0,1,0,0,0,7.4,14.713,1,0,...,0,0,1,0,1,0,0,0,0,1
2,Dunkirk,0,0,0,0,0,7.9,21.118,1,0,...,0,0,1,0,1,0,0,0,1,1
3,Phantom Thread,0,0,0,0,0,7.5,13.060,0,1,...,0,0,1,0,0,0,0,1,0,0
4,The Post,0,0,0,0,0,7.2,13.776,1,0,...,0,0,1,0,1,0,0,0,1,0
5,Call Me by Your Name,0,0,0,0,0,7.9,17.615,0,1,...,0,0,1,0,0,0,0,1,0,0
6,Lady Bird,0,0,0,0,0,7.4,17.003,0,1,...,1,0,1,0,0,0,0,0,0,0
7,"Three Billboards Outside Ebbing, Missouri",0,0,1,1,0,8.2,22.585,0,1,...,1,1,1,0,0,0,0,0,0,0
8,Get Out,0,0,0,0,0,7.7,24.057,0,1,...,0,0,0,0,0,1,1,0,1,0
